# Aula 4 - Parte Prática - Actor-Critic (A2C)

## Introdução

Nesse quarto notebook vamos estudar um dos algoritmos mais implementados da família dos *Policy Gradients*. Ao implementar o A2C, você terá contato com importantes conceitos utilizados em *Deep RL*. Em particular, utilizaremos pela primeira vez no curso redes neurais com *features* compartilhadas e ambientes vetorizados (i.e., paralelizados) para coleta de dados mais eficiente.


### Objetivos:

- Familiarizar-se com os componentes *Actor* e *Critic*
- Entender o papel da função Valor na estimativa truncada dos retornos
- Ter um primeiro contato com truques de implementação tipicamente utilizados e RL


### Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial.

In [1]:
!pip install gym[box2d]

In [2]:
!pip install pydot

In [3]:
import logging
from pprint import pprint

import gym
import pydot 
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from utils.agent import RLAgent, RandomAgent
from utils.memory import OnPolicyReplay
from utils.networks import build_actor_critic_network
import utils.runner
from utils.viz import *


tf.get_logger().setLevel("ERROR")     # ignore TensorFlow warnings
gym.logger.set_level(logging.ERROR)   # ignore OpenAI Gym warnings

## 0. LunarLander-v2

Para o notebook de hoje utilizaremos um outro problema do Gym que é mais desafiador que o CartPole.


> **Atenção**: para entender melhor a tarefa leia a documentação do LunarLander disponível em http://gym.openai.com/envs/LunarLander-v2/.

Execute o código abaixo para visualizar alguns episódios do agente aleatório para ter uma melhor ideia da tarefa:

In [4]:
env = gym.make("LunarLander-v2")

agent = RandomAgent(env.observation_space, env.action_space, None)

utils.runner.evaluate(agent, env, n_episodes=30, render=True)

>> episode = 0 / 30, total_reward =  -353.7703, episode_length = 109
>> episode = 1 / 30, total_reward =  -275.1834, episode_length = 107
>> episode = 2 / 30, total_reward =  -162.6296, episode_length = 98
>> episode = 3 / 30, total_reward =  -122.2536, episode_length = 94
>> episode = 4 / 30, total_reward =  -373.8535, episode_length = 100
>> episode = 5 / 30, total_reward =   -48.1675, episode_length = 70
>> episode = 6 / 30, total_reward =  -276.1076, episode_length = 111
>> episode = 7 / 30, total_reward =  -134.2783, episode_length = 123
>> episode = 8 / 30, total_reward =   -39.1867, episode_length = 125
>> episode = 9 / 30, total_reward =  -212.1545, episode_length = 66
>> episode = 10 / 30, total_reward =   -69.1504, episode_length = 89
>> episode = 11 / 30, total_reward =  -177.7158, episode_length = 71
>> episode = 12 / 30, total_reward =   -32.1991, episode_length = 65
>> episode = 13 / 30, total_reward =  -127.6573, episode_length = 60
>> episode = 14 / 30, total_reward =  

## 1. Ambientes vetorizados no Gym

Pela primeira vez no curso estaremos utilizando ambientes vetorizados, isto é, que emulam o comportamente de vários ambientes sendo executados em paralelo.

Execute o código abaixo e tente entender como o ambiente retornado pelo `gym.vector.make` se comporta.

In [5]:
env = gym.vector.make("LunarLander-v2", num_envs=4, asynchronous=True)

print(env.processes)

print(env.observation_space, env.single_observation_space)
print(env.action_space, env.single_action_space)

[<ForkProcess(Worker<AsyncVectorEnv>-0, started daemon)>, <ForkProcess(Worker<AsyncVectorEnv>-1, started daemon)>, <ForkProcess(Worker<AsyncVectorEnv>-2, started daemon)>, <ForkProcess(Worker<AsyncVectorEnv>-3, started daemon)>]
Box(4, 8) Box(8,)
Tuple(Discrete(4), Discrete(4), Discrete(4), Discrete(4)) Discrete(4)


In [6]:
n_steps_per_env = 3

observations = env.reset()

step = 0

for _ in range(n_steps_per_env):
    actions = env.action_space.sample()
    observations, rewards, dones, _ = env.step(actions)
    step += len(observations)

    print(f">> step = {step}")
    print(f"observations =\n{observations}")
    print(f"actions = {actions}")
    print(f"rewards = {rewards}")
    print(f"dones = {dones}")
    print()
    
print(f">> num_envs = {env.num_envs}, n_steps_per_env = {n_steps_per_env}, total timesteps = {step}")

>> step = 4
observations =
[[ 0.00501194  1.4019997   0.25347212 -0.21106076 -0.00573805 -0.0568253
   0.          0.        ]
 [ 0.00604897  1.404977    0.3059201  -0.14490628 -0.00692669 -0.068583
   0.          0.        ]
 [-0.00178061  1.3876907  -0.09474613 -0.5289163   0.00355007  0.05096614
   0.          0.        ]
 [-0.00424309  1.3905098  -0.20881315 -0.46624473  0.00301327  0.01007152
   0.          0.        ]]
actions = (0, 0, 1, 3)
rewards = [-1.36201025 -0.99585481 -1.73155803 -0.89997399]
dones = [False False False False]

>> step = 8
observations =
[[ 0.0075593   1.397739    0.25742063 -0.18937247 -0.00839923 -0.05322827
   0.          0.        ]
 [ 0.00916634  1.4011133   0.31755942 -0.1717561  -0.01268592 -0.11519512
   0.          0.        ]
 [-0.00279255  1.3751909  -0.106071   -0.55556864  0.00836617  0.09633093
   0.          0.        ]
 [-0.00622578  1.3800924  -0.199267   -0.46300352  0.00400537  0.01984377
   0.          0.        ]]
actions = (2, 3, 1, 2

## 2. Asychronous Actor-Critic (A2C)

<img src="img/a2c-algo.png" alt="A2C Algorithm" style="width: 700px;"/>


> **Observação**: para uma introdução mais intuitiva do A2C recomendamos o blog post https://sudonull.com/post/32170-Intuitive-RL-Reinforcement-Learning-Introduction-to-Advantage-Actor-Critic-A2C. 

### 2.1 Compartilhando informação entre *Actor* e *Critic*: *2-head model*

Na arquitetura A2C é comum implementar o Actor e o Critic em um mesmo modelo que compartilha parâmetros. Execute o código abaixo e tente interpretar a figura.

In [9]:
num_envs = 8
env = gym.vector.make("LunarLander-v2", num_envs=num_envs, asynchronous=True)

In [10]:
config = {
    "actor_critic_net": {
        "hidden_layers": [64, 64],
        "activation": "tanh"
    },
}

In [11]:
actor_critic_net = build_actor_critic_network(env.single_observation_space, env.single_action_space, config["actor_critic_net"])
tf.keras.utils.plot_model(actor_critic_net, show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


### 2.2 Otimizando a política e a função Valor via *joint loss*

$$
[\theta, \phi] \leftarrow [\theta, \phi] + \alpha \nabla_{\theta, \phi}(L_{actor}(\theta) + L_{critic}(\phi))
$$

onde $L_{actor}(\theta)$ e $L_{critic}(\phi)$ correspondem respectivamente ao *policy loss* e *mean squared error*:
$$
\begin{align*}
L_{actor}(\theta) &= - \frac{1}{K} \sum_{t=1}^K \log \pi_{\theta}(\mathbf{a}_t|\mathbf{s}_t) \hat{A}_t^{(n)} \\
L_{critic}(\phi) &= \frac{1}{K} \sum_{t=1}^K  (V_{\phi}(\mathbf{s}_t) - \hat{R}_t)^2
\end{align*}
$$

> **<font color="red">IMPORTANTE</color>**: Note que para o problema de regressão que precisamos resolver para aprender o *critic*, o retorno descontado $\hat{R}_t$ funciona como o *target* e, portanto, deve ser considerado uma "constante" para o TensorFlow. No exercício abaixo você precisará se lembrar disso!



### 2.3  Bônus de exploração: entropia da distribuição de ações

Além do *joint loss* adicionaremos também um outro termo à função objetivo a fim de incentivar o agente a continuar explorando novas ações.

$$
H(\pi_{\theta}) = \mathbb{E}_{\mathbf{s}, \mathbf{a} \sim \pi_{\theta}} \left [ - \log \pi_{\theta}(\mathbf{a}|\mathbf{s})\right]
$$

Tente entender no código abaixo onde e como esse bônus da entropia é implementado.

---

**<font color="red">EXERCÍCIO-PROGRAMA:</font>**

Nesse exercício você deverá implementar o *value function loss* no método `_joint_loss_fn` da classe abaixo.


> **Nota 1**: consulte a documentação do MSE em https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError. Será útil!

> **Nota 2**: você precisará utilizar a função `tf.stop_gradient` no cálculo do *target* na loss do Value Function. Isso permite que o TensorFlow não tente diferenciar o *target*. Veja nota <font color="red">IMPORTANTE</color> na Seção 2.2.

> **Nota 3**: se você tiver algum erro de "tipos", por exemplo, esperava-se `float32` em algum ponto do código mas o array do *NumPy* foi calculado como `float64` você poderá fazer *casting* manual de tipos usando o método de `astype("f")`.

In [12]:
class A2C(RLAgent):
    
    def __init__(self, obs_space, action_space, config):
        super().__init__(obs_space, action_space, config)
        
        self.memory = OnPolicyReplay()
        self.actor_critic = build_actor_critic_network(obs_space, action_space, config["actor_critic_net"])

        self.optimizer = tf.keras.optimizers.RMSprop(
            learning_rate=tf.keras.optimizers.schedules.PolynomialDecay(
                0.00083,
                decay_steps=config["total_timesteps"] / config["train_batch_size"],
                end_learning_rate=1e-4,
                power=1.0
            ))

    def act(self, obs):
        """
        Escolhe uma ação para ser tomada dada uma observação do ambiente.
        
        Args: 
            obs: observação do ambiente.
        
        Return:
            action: ação válida dentro do espaço de ações.
        """
        return self._act(obs).numpy()
        
    @tf.function
    def _act(self, obs):
        action_dist, _ = self.actor_critic(obs)
        return action_dist.sample()
    
    def observe(self, obs, action, reward, next_obs, done):
        """
        Registra na memória do agente uma transição do ambiente.

        Args:
            obs:            observação do ambiente antes da execução da ação.
            action:         ação escolhida pelo agente.
            reward (float): escalar indicando a recompensa obtida após a execução da ação.
            next_obs:       nova observação recebida do ambiente após a execução da ação.
            done (bool):    True se a nova observação corresponde a um estado terminal, False caso contrário.

        Return:
            None
        """
        self.memory.update(obs, action, reward, next_obs, done)

    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()
        weights = self.actor_critic.trainable_weights

        with tf.GradientTape() as tape:
            policy_loss, vf_loss, entropy_loss = self._joint_loss_fn(batch)
            loss = policy_loss + self.config["vf_loss_coeff"] * tf.cast(vf_loss, tf.float32) - self.config["entropy_coeff"] * entropy_loss
            gradients = tape.gradient(loss, weights)
    
        gradients = tuple(tf.clip_by_norm(grad, clip_norm=0.5) for grad in gradients)
        self.optimizer.apply_gradients(zip(gradients, weights))
      
        return {
            "policy_loss": policy_loss.numpy(),
            "vf_loss": vf_loss.numpy(),
            "entropy_loss": entropy_loss.numpy()
        }

    def _joint_loss_fn(self, batch):
        """
        Calcula a função loss do policy gradients para um `batch` de transições.
        
        Um `batch` agrega arrays n-dimensionais. Cada array (e.g., batch["states"],
        batch["actions"], batch["rewards"]) tem como primeiras duas dimensões o número
        de passos dados no ambiente vetorizado e o número de ambientes em paralelo. 
        Por exemplo, batch["states"][t][k] devolve um array correspondendo ao estado 
        no passo t devolvido pelo k-ésimo ambiente.

        Args:
            batch (Dict[str, np.ndarray]): dicionário para acesso às matrizes de 
                estados, ações, recompensas, próximos estados e flags de terminação. 
        
        Return:
            loss (tf.Tensor): surrogate loss conjunta da política, função valor e
                bônus de entropia.
        """
        states = batch["states"]
        actions = batch["actions"]
        rewards = batch["rewards"]
        next_states = batch["next_states"]
        dones = batch["dones"]

        n_steps = len(states)
        gamma = self.config["gamma"]
        lambda_ = self.config["lambda"]

        action_dists, values = self.actor_critic(states)
        _, last_value = self.actor_critic(next_states[-1:])

        values = tf.squeeze(tf.concat([values, last_value], axis=0))
        values, next_values = values[:-1], values[1:]

        deltas = rewards + gamma * (1 - dones) * next_values - values

        returns = np.empty_like(rewards)
        advantages = np.empty_like(rewards)

        returns[-1] = rewards[-1] + gamma * (1 - dones[-1]) * next_values[-1]
        advantages[-1] = deltas[-1]

        for t in reversed(range(n_steps - 1)):
            returns[t] = rewards[t] + gamma * (1 - dones[t]) * returns[t+1]
            advantages[t] = deltas[t] + (gamma * lambda_) * (1 - dones[t]) * advantages[t+1]

        log_probs = action_dists.log_prob(actions)

        policy_loss = - tf.reduce_sum(log_probs * tf.stop_gradient(advantages.astype("f")))
        
        # SEU CÓDIGO AQUI =====================================
        
        vf_loss = None
        
        env = gym.make('LunarLander-v2')
        for i_episode in range(10):
            observation = env.reset()
            for t in range(100):
                env.render()
                #print(observation)
                action = env.action_space.sample()
                observation, reward, done, info = env.step(action)
        env.close()
    
        # =====================================================
        entropy_loss = tf.reduce_mean(action_dists.entropy())

        return policy_loss, vf_loss, entropy_loss


Para treinar o seu agente A2C execute o código abaixo:

In [13]:
total_timesteps = 1_000_000

num_envs = 8
train_env = gym.vector.make("LunarLander-v2", num_envs=num_envs, asynchronous=True)

config = {
    "actor_critic_net": {
        "hidden_layers": [64, 64],
        "activation": "tanh"
    },
    "optimizer": {
        "class_name": "RMSprop",
        "config": {
            "learning_rate": 8e-4,
            "rho": 0.99
        }
    },
    "total_timesteps": total_timesteps,
    "train_batch_size": 40,
    "gamma": 0.995,
    "lambda": 1.0,
    "vf_loss_coeff": 0.25,
    "entropy_coeff": 1e-5
}

agent = A2C(train_env.single_observation_space, train_env.single_action_space, config)

timesteps, avg_total_rewards, losses = utils.runner.train(agent, train_env, total_timesteps)

plot_metrics(avg_total_rewards, losses)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

Para visualizar o agente treinado execute o código abaixo:

In [59]:
eval_env = gym.make("LunarLander-v2")
utils.runner.evaluate(agent, eval_env, n_episodes=10, render=True)

>> episode = 0 / 10, total_reward =  -144.9840, episode_length = 94
>> episode = 1 / 10, total_reward =  -531.1152, episode_length = 108
>> episode = 2 / 10, total_reward =   -85.4648, episode_length = 69
>> episode = 3 / 10, total_reward =  -118.7662, episode_length = 73
>> episode = 4 / 10, total_reward =  -116.4822, episode_length = 88
>> episode = 5 / 10, total_reward =  -390.7390, episode_length = 74
>> episode = 6 / 10, total_reward =   -89.6952, episode_length = 57
>> episode = 7 / 10, total_reward =  -205.2787, episode_length = 79
>> episode = 8 / 10, total_reward =  -568.8253, episode_length = 93
>> episode = 9 / 10, total_reward =   -28.7382, episode_length = 68


---

**<font color="red">PARABÉNS!</font>**


Se você conseguiu chegar até aqui, parabéns! Você conseguiu fazer funcionar uma primeira versão do A2C. Sabemos que isso não é uma tarefa fácil... :)

Esperamos que você tenha se familiarizado com as principais ideias por trás de um algoritmo de Deep RL e tenha vivenciado um pouco a dificuladade de treinar agentes de RL, mesmo para problemas aparentemente simples.